# 04 – V3 ResNet50 Transfer-Learning

We freeze the ImageNet-trained ResNet50 convolutional backbone, add a small classification head, and train on our waste-image dataset.

*Expect significant performance jump with minimal training time.*


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from src.data_loader   import load_images
from src.model_resnet  import build_resnet
from src.compile_utils import compile_model, early_stop
from src.plotting      import plot_history
from src.evaluation    import evaluate


## 1  Load & preprocess images
(Use the same 64 × 64 resize so every notebook is consistent.)


In [ ]:
folder_paths = [
    "../data/sample_images/recyclable",        # TODO replace
    "../data/sample_images/non_recyclable"     # TODO replace
]
class_names  = ["recyclable", "non-recyclable"]
target_size  = (64, 64)

X, y, _ = load_images(folder_paths, class_names, target_size)
le = LabelEncoder(); y_hot = to_categorical(le.fit_transform(y), 2)

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_hot, test_size=0.10, random_state=42, stratify=y_hot)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.22, random_state=42, stratify=y_temp)

X_train = X_train.astype("float32")/255.0
X_val   = X_val.astype("float32")/255.0
X_test  = X_test.astype("float32")/255.0

print("Train:", X_train.shape, "| Val:", X_val.shape, "| Test:", X_test.shape)




## 2  Build the ResNet50 + pool + dense head

* Freeze all convolutional layers.  
* Use **sigmoid** with `binary_crossentropy` because we still output two units.  
  (Alternative: softmax + categorical_crossentropy works as well.)*


In [ ]:
resnet = build_resnet(shape=(64,64,3), classes=2, trainable=False)
resnet = compile_model(resnet, lr=1e-3, loss="binary_crossentropy")
resnet.summary()


## 3  Train (frozen backbone)


In [ ]:
H = resnet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32,
    callbacks=[early_stop(patience=3)],
    verbose=2
)


## 4  Learning curves


In [ ]:
plot_history(H)


## 5  Evaluate on test set


In [ ]:
cm = evaluate(resnet, X_test, y_test, labels=le.classes_)


## 6  (OPTIONAL) Grad-CAM heatmap for qualitative explainability
Uncomment to visualise where ResNet focuses.


In [ ]:
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras import backend as K
# def grad_cam(input_img, model, cls, layer_name="conv5_block3_out"):
#     conv_layer = model.get_layer(layer_name)
#     heatmap_model = Model([model.inputs], [conv_layer.output, model.output])
#     with tf.GradientTape() as gtape:
#         conv_output, predictions = heatmap_model(np.array([input_img]))
#         loss = predictions[:, cls]
#     grads = gtape.gradient(loss, conv_output)[0]
#     heatmap = tf.reduce_mean(grads * conv_output[0], axis=-1)
#     heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
#     return heatmap


## 7  Discussion

* **Test accuracy** should exceed the custom CNN’s ~0.86 and land ~0.93–0.95.  
* Freezing backbone → only ~15 s training on CPU.  
* Next notebook (`05_v4_augmented_final.ipynb`) will combine data-augmentation with a lighter CNN to compare trade-offs.

If further accuracy needed, unfreeze **last 20 ResNet layers** and fine-tune at `1e-5` learning-rate.
